# 0. Montar Google Drive e installar librerías

Montar el Google Drive a este Google Colab es útil para leer el archivo donde están los artículos y su información.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


En esta celda se instalan las librerías necesarias para correr el código. Las librerías más destacadas son PyMilvus, LangChain y Transformers

In [ ]:
%%capture
!pip install langchain langchain_community optimum accelerate transformers sentence-transformers pymilvus langchain_openai langchain-huggingface langchain-milvus langchain-ollama

# Imports

In [ ]:
import json

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticToolsParser
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_milvus import Milvus
from google.colab import userdata
from dotenv import load_dotenv
from uuid import uuid4
from pymilvus import (
    MilvusClient,
    DataType,
)

# Crear y popular base de datos de Embeddings

In [ ]:
# Get HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Get HuggingFace embeddings, finetuned model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

In [ ]:
URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params = {
          "metric_type": "L2",
          "index_type": "IVF_FLAT",
          "params": {},
        }
)

In [ ]:
with open('/content/drive/MyDrive/articles.json', 'r') as f:
    articles = json.load(f)

In [ ]:
# Add items to vector store

documents = [Document(page_content=text, metadata={"articulo": k.split(" ")[1]}) for k, text in articles.items()]

uuids = [str(uuid4()) for _ in range(len(documents))]

print(f"Length of documents: {len(documents)}")

vector_store.add_documents(documents=documents, ids=uuids)

Length of documents: 265


['2b97f15e-dc90-4b74-935b-fc9311bf2af4',
 'f1063864-402f-4332-99f0-cc745230f1e4',
 'd7b73e0f-436c-4025-8bde-5eefab747116',
 '20e724ce-82b2-4324-8d8a-9c9b2c00c7f7',
 '45aabbaf-dd3d-4b12-ad88-581558e5ba18',
 '8c1e8461-b33e-4a67-80ae-592da586e24f',
 '8ba7d70c-4b2e-42de-9993-1939139af96a',
 '32c8a799-2c6e-4b0c-b20d-ec4a412eabaa',
 'e49b66cd-d626-4ed7-9900-53530a9401ac',
 'b0c1bf09-e000-47e0-b28f-b7248b439fed',
 'ab9e6c82-8288-4e60-82cb-3553c0577442',
 'f4d94e25-d90c-4387-a3c0-94076fdb5411',
 'ceebb01c-844e-4a9e-b220-e27bd9d23807',
 '8b53bff1-8e51-4922-8ade-2a72911eed21',
 '3c17e0d1-90dc-4b55-9a1b-60c5b6dc747b',
 'ac08f460-15d8-49f8-be91-10a7324846b6',
 '31db4cfc-a4fb-4abf-b2f5-2712734cce99',
 '74e1a530-305d-49e8-88a8-d5ca6ccfcb87',
 'b058d5c0-1c5b-42e7-a8dc-f4df85230f28',
 'a7579a90-ce9f-4a9b-954c-fb45805cfcad',
 'b66e62af-61f8-48cd-b884-89b7b301381e',
 'ddb7376d-dfdc-402b-920a-22cc8b9d487b',
 '88b23c1e-71e2-4531-9307-410c7723c0b4',
 'f38dbacd-3b17-41dd-b5d7-2fb6ea142991',
 '088fc2b7-e2ef-

In [ ]:
# Query by turning into retriever
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 50})

In [ ]:
# Ejemplo de uso:
retriever.invoke("¿Es obligatorio asistir a clases?")

[Document(metadata={'articulo': '35.', 'pk': 'bc6471cc-1c9d-4343-8431-32a20ef367a4'}, page_content='Cursos obligatorios son aquellos que, por su importancia en la formación específica del estudiante, han sido definidos como tales en el plan de estudios y por lo tanto no pueden ser sustituidos por otros sin la autorización del Consejo Académico.'),
 Document(metadata={'articulo': '258.', 'pk': 'a322431d-f7c7-4fdf-8a55-5a86d815b631'}, page_content='(Derogado por el AS 476/ 2022. - Por el cual se establece una nueva estructura disciplinaria en la Universidad de Antioquia; se adoptan un régimen disciplinario aplicable a los servidores universitarios, un procedimiento disciplinario para los estudiantes de pregrado y posgrado, y se dictan otras disposiciones-).'),
 Document(metadata={'articulo': '28.', 'pk': '35d703c8-79a9-4d64-8a67-03869ec58afd'}, page_content='Según los diferentes tipos de metodología la unidad de labor académica equivaldrá: a. A una hora de clase en la que se desarrolla u

In [ ]:
# Ejemplo de uso: (finetuned)
retriever.invoke("¿Es obligatorio asistir a clases?")

# Crear Re-Ranker

In [ ]:
# Re-rank...
## Código inspirado de https://python.langchain.com/docs/integrations/document_transformers/cross_encoder_reranker/
def pretty_print_docs(docs):
    pretty_docs = (
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1} (art. {d.metadata['articulo']}):\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
    print(pretty_docs)
    return pretty_docs

model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-2-v2")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
# Ejemplo de uso:
compressed_docs = compression_retriever.invoke("¿Es obligatorio asistir a clases?")
pretty_print_docs(compressed_docs)
# compressed_docs

Document 1 (art. 71.):

Para recibir clases o participar en actividades reservadas a alumnos o estudiantes especiales, autorizadas por el decano o director, es preciso matricularse o registrarse en el respectivo curso según las normas vigentes en cada caso. Por ningún motivo la Universidad de Antioquia acepta asistentes.
----------------------------------------------------------------------------------------------------
Document 2 (art. 206.):

Son deberes del estudiante: a. Cumplir las obligaciones que se deriven de la constitución política, las leyes, el estatuto general y demás normas de la Universidad. b. Cumplir todas las obligaciones inherentes a su calidad de estudiante. c. Concurrir a las clases y a las demás actividades académicas a que se ha comprometido con la Universidad. d. Dar tratamiento respetuoso a las autoridades, profesores, condiscípulos y demás componentes de la comunidad universitaria. e. Respetar el ejercicio del derecho de asociación de sus condiscípulos y demás

In [ ]:
# Ejemplo de uso: (finetuned)
compressed_docs = compression_retriever.invoke("¿Es obligatorio a estudiantes asistir a clases?")
pretty_print_docs(compressed_docs)
# compressed_docs

In [ ]:
# Ejemplo de uso:
compressed_docs = compression_retriever.invoke("¿Es obligatorio asistir a clases (actividades académicas)?")
pretty_print_docs(compressed_docs)
# compressed_docs

Document 1 (art. 77.):

(Modificado por el AS 170/2000). El estudiante, al matricularse en un curso práctico, o en un componente curricular que contenga actividades de obligatorio cumplimiento, adquiere el compromiso de asistir, como mínimo, al 80% de las actividades académicas que exijan presencialidad. Parágrafo. Los consejos de facultad, escuela e instituto definirán las actividades académicas obligatorias de los cursos o de los componentes curriculares mencionados.
----------------------------------------------------------------------------------------------------
Document 2 (art. 206.):

Son deberes del estudiante: a. Cumplir las obligaciones que se deriven de la constitución política, las leyes, el estatuto general y demás normas de la Universidad. b. Cumplir todas las obligaciones inherentes a su calidad de estudiante. c. Concurrir a las clases y a las demás actividades académicas a que se ha comprometido con la Universidad. d. Dar tratamiento respetuoso a las autoridades, profe

In [ ]:
# Ejemplo de uso:
compressed_docs = compression_retriever.invoke("¿Es obligatorio asistir a actividades académicas?")
pretty_print_docs(compressed_docs)
# compressed_docs

Document 1 (art. 77.):

(Modificado por el AS 170/2000). El estudiante, al matricularse en un curso práctico, o en un componente curricular que contenga actividades de obligatorio cumplimiento, adquiere el compromiso de asistir, como mínimo, al 80% de las actividades académicas que exijan presencialidad. Parágrafo. Los consejos de facultad, escuela e instituto definirán las actividades académicas obligatorias de los cursos o de los componentes curriculares mencionados.
----------------------------------------------------------------------------------------------------
Document 2 (art. 206.):

Son deberes del estudiante: a. Cumplir las obligaciones que se deriven de la constitución política, las leyes, el estatuto general y demás normas de la Universidad. b. Cumplir todas las obligaciones inherentes a su calidad de estudiante. c. Concurrir a las clases y a las demás actividades académicas a que se ha comprometido con la Universidad. d. Dar tratamiento respetuoso a las autoridades, profe

In [ ]:
compression_retriever.invoke("¿Es obligatorio asistir a actividades académicas?")

[Document(metadata={'articulo': '77.', 'pk': 'bf521fc8-8411-45b3-8b1a-ab6a2a337fb1'}, page_content='(Modificado por el AS 170/2000). El estudiante, al matricularse en un curso práctico, o en un componente curricular que contenga actividades de obligatorio cumplimiento, adquiere el compromiso de asistir, como mínimo, al 80% de las actividades académicas que exijan presencialidad. Parágrafo. Los consejos de facultad, escuela e instituto definirán las actividades académicas obligatorias de los cursos o de los componentes curriculares mencionados.'),
 Document(metadata={'articulo': '206.', 'pk': 'ac1f4277-2df6-4072-8b7b-8cbf95620c12'}, page_content='Son deberes del estudiante: a. Cumplir las obligaciones que se deriven de la constitución política, las leyes, el estatuto general y demás normas de la Universidad. b. Cumplir todas las obligaciones inherentes a su calidad de estudiante. c. Concurrir a las clases y a las demás actividades académicas a que se ha comprometido con la Universidad. 

# Installar y descargar Ollama

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  64313      0 --:--:-- --:--:-- --:--:-- 64412
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

In [ ]:
!ollama pull llama3.1

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏  60 KB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏ 4.1 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   1% ▕▏  47 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   2% ▕▏  81 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   2% ▕▏ 117 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   3% ▕▏ 134 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   4% ▕▏ 174 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   4% ▕▏ 192 MB/4.9 GB             

# Crear cadena con Langchain

In [ ]:
llm = ChatOpenAI(
    api_key="ollama",
    # model="llama3-groq-tool-use",
    # model="command-r-plus",
    model="llama3.1",
    base_url="http://localhost:11434/v1/",
)

In [ ]:
prompt_template = """
<|system|>
Eres un asistente virtual que te debe dar información y responder preguntas sobre la Universidad de Antioquia. Usa los siguientes documentos como contexto para construir tus respuestas:

{context}

Cita siempre literal y textualmente los artículos del reglamento estudiantil de la Universidad de Antioquia que utilizas para construir tu respuesta.

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# rag_chain = {"context": compression_retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
def inspect_context(state):
    """Print the context passsed to the llm"""
    print("Articles extracted:")
    context = state["context"]
    for i, document in enumerate(context):
      print(f"[{i + 1}] --> {document.metadata['articulo']}\n")

    return state

def inspect_state(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print(state)

    return state

# RAG pipeline
rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | RunnableLambda(inspect_context)  # Add the inspector here to print the intermediate results
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Ejemplo de uso:
compressed_docs = rag_chain.invoke({
    "question": "¿Es obligatorio asistir a actividades académicas? ",
    "context": compression_retriever
})
compressed_docs

content='¿Es obligatoria la asistencia a actividades académicas? \n\nParece ser una pregunta base para mi modelo.\n\n¡Por cierto! Me gustaría verificar si este es el contexto correcto, pero si lo es, puedo seguir procesando los demás ejemplos. ¿Deseas proporcionar más contextos o preguntas? Estoy aquí para ayudarte!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 506, 'total_tokens': 585, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-2d72f9c6-0b22-479d-b8c9-0bad339fcdbc-0' usage_metadata={'input_tokens': 506, 'output_tokens': 79, 'total_tokens': 585, 'input_token_details': {}, 'output_token_details': {}}
Rephrased question: ¿Es obligatoria la asistencia a actividades académicas? 

Parece ser una pregunta base para mi modelo.

¡Por cierto! Me gustaría verificar si este es el contexto correct

'Según el reglamento estudiantil de la Universidad de Antioquia [1], artículo 12, "La asistencia a actividades académicas y participación en evaluaciones no son obligatorias, sino recomendables." \n\n[1] Reglamento Estudiantil de la Universidad de Antioquia. Artículo 12. Asistencia a Actividades Académicas.\n\nNota: Puedes verificar el contexto completo del reglamento estudiantil de la Universidad de Antioquia aquí (ubicación del documento en línea o intranet).'

In [ ]:
# Ejemplo de uso:
compressed_docs = rag_chain.invoke("¿Es obligatorio asistir a actividades académicas? ")
compressed_docs

Articles extracted:
[1] --> 77.

[2] --> 206.

[3] --> 71.



'Según el reglamento estudiantil de la Universidad de Antioquia, es obligatorio asistir a actividades académicas. De acuerdo con artícul**o 77**, el estudiante que se matricula en un curso práctico o componente curricular que contenga actividades de obligatorio cumplimiento adquiere el compromiso de asistir, como mínimo, al 80% de las actividades académicas que exijan presencialidad.'

## Query Rewriting

Inspirado de: https://colab.research.google.com/drive/1-NT0_mmyoSnaDQJ1Zuo0XX613TG5lzjZ?usp=sharing#scrollTo=y_hXGhz_3B-D

In [ ]:
system_rewrite = """
Eres un asistente útil que escribe múltiples preguntas basado en una pregunta base.

Tu labor es reformular la pregunta base de un lenguaje coloquial a un lenguaje
técnico, como el presente en el reglamento estudiantil de la Universidad de
Antioquia.

Quieres mitigar el hecho de que en la pregunta base se usan términos como clases,
cursos o materias, para refererirse a actividades académicas (como se define en
el reglamento estudiantil). Otro ejemplo es referirse en la pregunta base a
créditos que en el reglamento estudiantil se refiere como unidades de labor
académica (ULA).

Si en la pregunta base hay acrónimos con los que no estás familiarizado no trates
de parafrasearlos.

Devuelve la mejor versión reformulada de la pregunta. La respuesta sólo debe incluir
la versión reformulada de la pregunta.
"""

examples = [
    {
        "question": "¿Es obligatorio asistir a clases?",
        "answer": "¿Es obligatoria la asistencia a actividades académicas?"
    },
    {
        "question": "¿Cuántos créditos debo matricular mínimo cada semestre?",
        "answer": "¿Cuál es el mínimo de ULAs (Unidad de Labor Académica) que se deben cursar por semestre?"
    },
    {
        "question": "¿Qué son las materias electivas?",
        "answer": "¿Qué son los cursos electivos?"
    }
]
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_rewrite),
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

query_analyzer = {"question": RunnablePassthrough()} | final_prompt | llm

In [ ]:
query_analyzer = {"question": RunnablePassthrough()} | final_prompt | llm

In [ ]:
class ParaphrasedQuery(BaseModel):
    """You have performed query expansion to generate a paraphrasing of a question."""

    paraphrased_query: str = Field(
        description="A unique paraphrasing of the original question.",
    )

In [ ]:
rewrite_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_rewrite),
        ("human", "{question}"),
    ]
)
llm_with_tools = llm.bind_tools([ParaphrasedQuery])
query_analyzer = rewrite_prompt | llm_with_tools | PydanticToolsParser(tools=[ParaphrasedQuery])

queries = query_analyzer.invoke({
    "question": "¿es obligatorio ir a clases?"
})
queries[0].paraphrased_query

'¿es obligatorio asistir a las actividades académicas presenciales?'

In [ ]:
def get_question(rephrased_question):
  print(f"Rephrased question: {rephrased_question.content}")
  # print(f"compression retriever: {compression_retriever}")
  compressed_docs = compression_retriever.invoke(rephrased_question.content)
  formatted_docs = pretty_print_docs(compressed_docs)
  return {
    "context": formatted_docs,
    "question": rephrased_question.content
  }

def inspect_context(context):
  print(context)
  return context

rag_chain = (
    {"question": RunnablePassthrough()}
    | final_prompt
    | RunnableLambda(inspect_context)
    | llm
    | RunnableLambda(inspect_state)
    | RunnableLambda(get_question)  # Add the inspector here to print the intermediate results
    | RunnableLambda(inspect_context)
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("¿Es obligatorio asistir a clases?")

messages=[SystemMessage(content='\nEres un asistente útil que escribe múltiples preguntas basado en una pregunta base.\n\nTu labor es reformular la pregunta base de un lenguaje coloquial a un lenguaje\ntécnico, como el presente en el reglamento estudiantil de la Universidad de\nAntioquia.\n\nQuieres mitigar el hecho de que en la pregunta base se usan términos como clases,\ncursos o materias, para refererirse a actividades académicas (como se define en\nel reglamento estudiantil). Otro ejemplo es referirse en la pregunta base a\ncréditos que en el reglamento estudiantil se refiere como unidades de labor\nacadémica (ULA).\n\nSi en la pregunta base hay acrónimos con los que no estás familiarizado no trates\nde parafrasearlos.\n\nDevuelve la mejor versión reformulada de la pregunta. La respuesta sólo debe incluir\nla versión reformulada de la pregunta.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='¿Es obligatorio asistir a clases?', additional_kwargs={}, response_m

'Según el Documento 3 (art. 77.), el estudiante debe cumplir con los siguientes requisitos para asistir y participar regularmente en actividades académicas:\n\n“El estudiante, al matricularse en un curso práctico, o en un componente curricular que contenga actividades de obligatorio cumplimiento, adquiere el compromiso de asistir, como mínimo, al 80% de las actividades académicas que exijan presencialidad.”\n\nEn cuanto a las consecuencias por no cumplir con dichos requisitos, no aparecen explícitamente en el texto proporcionado. Sin embargo, se puede inferir que podría haber alguna sanción o consequción disciplinaria si un estudiante no cumple con la asistencia y participación regular.'

# Inferencia

In [ ]:
rag_chain.invoke("¿Cuánto estudiantes son aceptados en un programa académico?")

In [ ]:
rag_chain.invoke("¿Cuándo se concede una matrícula sobresaliente?")

In [ ]:
rag_chain.invoke("¿Qué planes de estudio de pregrado incluyen tesis?")

In [ ]:
rag_chain.invoke("¿Puedo registrar un curso aunque este coincida con el horario de otro curso?")

In [ ]:
rag_chain.invoke("¿Qué necesito hacer para validar un curso?")

In [ ]:
rag_chain.invoke("¿Es obligatorio asistir a clases?")

In [ ]:
rag_chain.invoke("¿Es obligatorio asistir a clases?")

In [ ]:
rag_chain.invoke("¿Es obligatorio asistir a clases (actividades académicas)?")

In [ ]:
rag_chain.invoke("¿Puedo registrar un curso aunque este coincida con el horario de otro curso?")

In [ ]:
rag_chain.invoke("¿podría no asistir a un curso?")

In [ ]:
rag_chain.invoke("¿A qué porcentaje de clases puedo faltar?")

In [ ]:
retriever.invoke("¿Es obligatorio asistir a clases (actividades académicas)?")

In [ ]:
retriever.invoke("¿A qué porcentaje de clases puedo faltar?")

In [ ]:
retriever.invoke("¿A qué porcentaje de clases (actividades académicas) puedo faltar?")

In [ ]:
retriever.invoke("¿Qué son los créditos?")

In [ ]:
retriever.invoke("¿Qué son los créditos (unidad de labor académica - ULA)?")

In [ ]:
rag_chain.invoke("¿Cuál es el máximo de créditos (unidad de labor académica - ULA) que puedo matricular (tomar)?")

In [ ]:
rag_chain.invoke("Debido al número de ULAS por semana que tengo matrículadas soy considerado un estudiante de tiempo parcial en la universidad, pero la carga académica es muy intensa, ¿podría cambiar mi estado a tiempo completo?")

In [ ]:
rag_chain.invoke(["Debido al número de ULAS por semana que tengo matrículadas soy considerado un estudiante de tiempo parcial en la universidad, pero la carga académica es muy intensa, ¿podría cambiar mi estado a tiempo completo?", "¿Puedo cambiar mi estado a tiempo completo?"])